In [1]:
! pip install "autogen-ext[openai]"
! pip install autogen-agentchat
! pip install azure-ai-projects 
! pip install azure-identity

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: C:\Users\pranabp\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: C:\Users\pranabp\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: C:\Users\pranabp\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: C:\Users\pranabp\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage
from autogen_core import CancellationToken
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient

In [3]:

from azure.identity import DefaultAzureCredential, get_bearer_token_provider

In [4]:
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import BingGroundingTool

In [5]:
token_provider = get_bearer_token_provider(DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default")

In [ ]:
az_model_client = AzureOpenAIChatCompletionClient(
    azure_deployment="gpt-4",
    api_version="2024-12-01-preview",
    model = "gpt-4",
    azure_endpoint="Your Azure OpenAI Service endpoint from Azure AI Foundry",
    # azure_ad_token_provider=token_provider,  # Optional if you choose key-based authentication.
    api_key="Your API Key from Azure AI Foundry", # For key-based authentication.
)

In [ ]:
project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str='Your Project connection string from Azure Foundry here',
)

In [ ]:
bing_connection = project_client.connections.get(
    connection_name='Grounding with Bing Search connection name from Azure Foundry here'
)

conn_id = bing_connection.id

In [9]:
async def web_ai_agent(query: str) -> str:
    print("This is Bing for Azure AI Agent Service .......")
    bing = BingGroundingTool(connection_id=conn_id)
    with project_client:
        agent = project_client.agents.create_agent(
            model="gpt-4",
            name="my-assistant",
            instructions="""        
                You are a web search agent.
                Your only tool is search_tool - use it to find information.
                You make only one search call at a time.
                Once you have the results, you never do calculations based on them.
            """,
            tools=bing.definitions,
            headers={"x-ms-enable-preview": "true"}
        )
        print(f"Created agent, ID: {agent.id}")

        # Create thread for communication
        thread = project_client.agents.create_thread()
        print(f"Created thread, ID: {thread.id}")

        # Create message to thread
        message = project_client.agents.create_message(
            thread_id=thread.id,
            role="user",
            content=query,
        )
        print(f"SMS: {message}")
        # Create and process agent run in thread with tools
        run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=agent.id)
        print(f"Run finished with status: {run.status}")

        if run.status == "failed":
            print(f"Run failed: {run.last_error}")

        # Delete the assistant when done
        # project_client.agents.delete_agent(agent.id)
        # print("Deleted agent")

        # Fetch and log all messages
        messages = project_client.agents.list_messages(thread_id=thread.id)
        print("Messages:"+ messages["data"][0]["content"][0]["text"]["value"])
    return messages["data"][0]["content"][0]["text"]["value"]



In [10]:
bing_search_agent = AssistantAgent(
    name="assistant",
    model_client=az_model_client,
    tools=[web_ai_agent],
    system_message="Use tools to solve tasks.",
)

In [11]:
bing = BingGroundingTool(connection_id=conn_id)

In [12]:
async def assistant_run() -> None:
    response = await bing_search_agent.on_messages(
            [TextMessage(content="What is Azure OpenAI?", source="user")],
            cancellation_token=CancellationToken(),
    )
#     print(response.inner_messages)
    print(response.chat_message)

In [13]:
await assistant_run()

C:\Users\pranabp\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\autogen_agentchat\agents\_assistant_agent.py:870: UserWarning: Resolved model mismatch: gpt-4-0613 != gpt-4-turbo-2024-04-09. Model mapping in autogen_ext.models.openai may be incorrect. Set the model to gpt-4-turbo-2024-04-09 to enhance token/cost estimation and suppress this warning.
  model_result = await model_client.create(


This is Bing for Azure AI Agent Service .......
Created agent, ID: asst_Nv4mzuIFRfnbW2Tpleco471m
Created thread, ID: thread_ajpImjpFoQbI1B49zcbvlMIi
SMS: {'id': 'msg_dlofPukTmLabXiJbv6q12glP', 'object': 'thread.message', 'created_at': 1743632548, 'assistant_id': None, 'thread_id': 'thread_ajpImjpFoQbI1B49zcbvlMIi', 'run_id': None, 'role': 'user', 'content': [{'type': 'text', 'text': {'value': 'What is Azure OpenAI?', 'annotations': []}}], 'attachments': [], 'metadata': {}}
Run finished with status: RunStatus.COMPLETED
Messages:Azure OpenAI is a collaboration between Microsoft and OpenAI that provides enterprise users with access to OpenAI’s advanced artificial intelligence models, including the notable GPT (Generative Pre-trained Transformer) and DALL-E models, through Microsoft's Azure cloud services. This partnership allows businesses and organizations to integrate and deploy AI technologies such as natural language processing, text generation, and image generation within their opera